In [37]:
import numpy as np

from torch.utils.data import DataLoader
from torchvision import datasets

from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as Trans
import torch.nn.init as init


In [38]:
BATCH_SIZE = 32
NUM_WORKERS = 0

EPOCHS = 10

In [39]:
transform = Trans.ToTensor()


train_dataset = datasets.MNIST(
    root='datasets',
    train=True,
    transform=transform,
    download=True
)

test_dataset = datasets.MNIST(
    root='datasets',
    train=False,
    transform=transform,
    download=True
)

train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
)

test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
)

In [40]:
class MyNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool_1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.pool_2 = nn.MaxPool2d(2, 2)

        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool_1(F.relu(self.conv1(x)))
        x = self.pool_2(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

CELoss = nn.CrossEntropyLoss()
net = MyNet()

optimizer = torch.optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

In [41]:
def train(net, optimizer, criterion):
    running_loss = 0
    for images, labels in train_dataloader:
        
        optimizer.zero_grad()

        output = net(images)

        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss
    with torch.no_grad():
        train_loss = running_loss / len(train_dataloader)
    return train_loss


def valid(net, criterion):
    running_loss = 0
    correct_total = 0
    with torch.no_grad():
        for images, labels in test_dataloader:
            output = net(images)

            loss = criterion(output, labels)
            running_loss += loss

            pred = output.max(dim = 1, keepdim=True)[1]
            correct_total += pred.eq(labels.view_as(pred)).sum()
            
        precison = correct_total / len(test_dataloader.dataset)
        valid_loss = running_loss / len(test_dataloader)
        return valid_loss, precison

In [42]:
for epoch in (pbar := tqdm(range(EPOCHS))):
    train_loss = train(net, optimizer, CELoss)
    valid_loss, prec = valid(net, CELoss)

    print(f"[{epoch}] train/valid loss: {train_loss:.4f}/{valid_loss:.4f} prec: {prec:.4f}")
    pbar.set_description(f"train/valid loss: {train_loss:.4f}/{valid_loss:.4f} prec: {prec:.4f}")

  0%|          | 0/10 [00:00<?, ?it/s]

[0] train/valid loss: 0.6014/0.1141 prec: 0.9633


KeyboardInterrupt: 

: 